# Thesis experiments

In [11]:
import numpy
from cade.metrics.comparative import moving_lncs2, lncs2, get_neighbors_set
from gensim.models.word2vec import Word2Vec
from scipy.spatial.distance import cosine
from scipy.stats import entropy
from scipy.special import softmax
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    classification_report
)
from scipy.stats import spearmanr
from tabulate import tabulate
from config import CURRENT_EXP_DIR, config, get_logger, log_config


## Load language models and groundtruth

In [12]:
def get_models(lang: str):
    model1 = Word2Vec.load(
        CURRENT_EXP_DIR.split("_")[0]
        + "_0"
        + "/model/"
        + lang
        + "/corpus1.model"
    )
    model2 = Word2Vec.load(
        CURRENT_EXP_DIR.split("_")[0]
        + "_0"
        + "/model/"
        + lang
        + "/corpus2.model"
    )
    return model1, model2

def get_gt(lang: str, binary=True):
    binary_truth = numpy.loadtxt(
        "./data/"
        + lang
        + "/semeval2020_ulscd_"
        + lang[:3]
        + "/truth/" + ("binary" if binary else "graded") + ".txt",
        dtype=str,
        delimiter="\t",
    )
    return binary_truth

### Redefined LNCS2 with softmax and normalized entropy

In [31]:
def lncs2_entropy(word, m1, m2, topn, verbose=False):
    """
    https://www.aclweb.org/anthology/D16-1229/

    :param word:
    :param m1:
    :param m2:
    :param topn:
    :return:
    """

    words_m1 = list(get_neighbors_set(word, m1, topn))
    words_m2 = list(get_neighbors_set(word, m2, topn))

    vec_1 = []
    vec_2 = []
    avg = 0
    mean = False

    # Cosine similarity between "word" and every word in its m1-neighbour
    # within the m1 space
    for wtest in words_m1:
        vec_1.append(1 - cosine(m1.wv[word], m1.wv[wtest]))

    # Cosine similarity between "word" and every word in its m2-neighbour
    # within the m1 space
    for wtest in words_m2:
        if wtest not in m1.wv.vocab:
            if not mean:
                # Represent OOV words in m1 space empirically with its mean
                avg = numpy.average(m1[m1.wv.vocab], axis=0)
                mean = True
            vec_1.append(1 - cosine(m1[word], avg))
        else:
            vec_1.append(1 - cosine(m1.wv[word], m1.wv[wtest]))

    avg = 0
    mean = False

    # Cosine similarity between "word" and every word in its m1-neighbour
    # within the m2 space
    for wtest in words_m1:
        if wtest not in m2.wv.vocab:
            if not mean:
                # Represent OOV words in m1 space empirically with its mean
                avg = numpy.average(m2[m2.wv.vocab], axis=0)
                mean = True
            vec_2.append(1 - cosine(m2[word], avg))
        else:
            vec_2.append(1 - cosine(m2.wv[word], m2.wv[wtest]))

    # Cosine similarity between "word" and every word in its m2-neighbour
    # within the m2 space
    for wtest in words_m2:
        vec_2.append(1 - cosine(m2.wv[word], m2.wv[wtest]))

    if verbose:
        print(vec_1)
    vec_1 = numpy.exp(-numpy.array(vec_1))
    vec_1 = vec_1 / vec_1.sum()
    if verbose:
        print(vec_1)
    vec_2 = numpy.exp(-numpy.array(vec_2))
    vec_2 = vec_2 / vec_2.sum()

    return entropy(vec_1, vec_2)

### English (Hyper on ACC: thr=0.7329, t=0.6107, NN=36)

In [37]:
lang = "english"
# Load models
model1, model2 = get_models(lang)
# Load binary truths
binary_truth = get_gt(lang)
# Task 1 - Binary Classification
table = []
predictions = []
i = 0
verbose = True
for word in binary_truth[:, 0]:
    prediction = lncs2_entropy(word, model1, model2, 30, verbose)
    predictions.append(prediction)
    table.append([word, str(binary_truth[i, 1]), str(prediction)])
    i += 1
    if verbose:
        verbose = False
print(tabulate(table, headers=["Word","Truth", "Prediction"]))
""" print("CLassification score for " + lang)
print(
    "\n"
    + classification_report(
        binary_truth[:, 1].astype(float),
        numpy.array(predictions),
        target_names=["class 0 (stable)", "class 1 (change)"],
    )
)
# Load scores truths
score_truth = get_gt(lang, binary=False)
table = []
# Task 2 - Semantic Shift Score
scores = []
i = 0
for word in score_truth[:, 0]:
    score = 1 - moving_lncs2(word, model1, model2, 36, 0.6107)
    scores.append(score)
    table.append([word, str(score_truth[i, 1]), str(score)])
    i += 1
print(tabulate(table, headers=["Word","Truth", "Rank"]))
rho, _ = spearmanr(scores, score_truth[:, 1], nan_policy="raise")
print("CLassification score for " + lang)
print("Spearman score for " + lang + ": " + str(rho)) """

[0.666697084903717, 0.6352203488349915, 0.5186610221862793, 0.8356345891952515, 0.5687299966812134, 0.49635517597198486, 0.6045973300933838, 0.5100687742233276, 0.5714151859283447, 0.6168639659881592, 0.5856351852416992, 0.5778291821479797, 0.5923551917076111, 0.5167604684829712, 0.5109189748764038, 0.5468693375587463, 0.5224267244338989, 0.5034369230270386, 0.4993384778499603, 0.4993321895599365, 0.5561960935592651, 0.5072870254516602, 0.5236865282058716, 0.6097083687782288, 0.5127843618392944, 0.5087453722953796, 0.5462506413459778, 0.5315718054771423, 0.5868595242500305, 0.5237288475036621, 0.25663283467292786, 0.25663283467292786, 0.5186610221862793, 0.25663283467292786, 0.8356345891952515, 0.41574421525001526, 0.2621910870075226, 0.3715507686138153, 0.10474476218223572, 0.6045973300933838, 0.06434650719165802, 0.25663283467292786, 0.6168639659881592, 0.21564006805419922, 0.09363753348588943, 0.30701854825019836, 0.25663283467292786, 0.4748297929763794, 0.31637004017829895, 0.08262

' print("CLassification score for " + lang)\nprint(\n    "\n"\n    + classification_report(\n        binary_truth[:, 1].astype(float),\n        numpy.array(predictions),\n        target_names=["class 0 (stable)", "class 1 (change)"],\n    )\n)\n# Load scores truths\nscore_truth = get_gt(lang, binary=False)\ntable = []\n# Task 2 - Semantic Shift Score\nscores = []\ni = 0\nfor word in score_truth[:, 0]:\n    score = 1 - moving_lncs2(word, model1, model2, 36, 0.6107)\n    scores.append(score)\n    table.append([word, str(score_truth[i, 1]), str(score)])\n    i += 1\nprint(tabulate(table, headers=["Word","Truth", "Rank"]))\nrho, _ = spearmanr(scores, score_truth[:, 1], nan_policy="raise")\nprint("CLassification score for " + lang)\nprint("Spearman score for " + lang + ": " + str(rho)) '

### German (Hyper on ACC: thr=0.5, t=0.7930, NN=18)

In [4]:
lang = "german"
# Load models
model1, model2 = get_models(lang)
# Load binary truths
binary_truth = get_gt(lang)
# Task 1 - Binary Classification
table = []
predictions = []
i = 0
for word in binary_truth[:, 0]:
    prediction = (
        0
        if moving_lncs2(word, model1, model2, 18, 0.7930) >= 0.5
        else 1
    )
    predictions.append(prediction)
    table.append([word, str(binary_truth[i, 1]), str(prediction)])
    i += 1
print(tabulate(table, headers=["Word","Truth", "Prediction"]))
print("CLassification score for " + lang)
print(
    "\n"
    + classification_report(
        binary_truth[:, 1].astype(float),
        numpy.array(predictions),
        target_names=["class 0 (stable)", "class 1 (change)"],
    )
)
# Load scores truths
score_truth = get_gt(lang, binary=False)
table = []
# Task 2 - Semantic Shift Score
scores = []
i = 0
for word in score_truth[:, 0]:
    score = 1 - moving_lncs2(word, model1, model2, 18, 0.7930)
    scores.append(score)
    table.append([word, str(score_truth[i, 1]), str(score)])
    i += 1
print(tabulate(table, headers=["Word","Truth", "Rank"]))
rho, _ = spearmanr(scores, score_truth[:, 1], nan_policy="raise")
print("CLassification score for " + lang)
print("Spearman score for " + lang + ": " + str(rho))

Word                  Truth    Prediction
------------------  -------  ------------
abbauen                   1             1
abdecken                  1             0
abgebrüht                 0             1
Abgesang                  1             1
Ackergerät                0             0
Armenhaus                 0             1
artikulieren              1             1
aufrechterhalten          0             1
Ausnahmegesetz            0             0
ausspannen                1             1
beimischen                0             0
Dynamik                   1             1
Einreichung               0             0
Eintagsfliege             0             1
Engpaß                    1             1
Entscheidung              0             0
Festspiel                 0             0
Frechheit                 0             0
Fuß                       0             0
Gesichtsausdruck          0             0
Knotenpunkt               1             1
Kubikmeter                0       

### Latin (Hyper on ACC: thr=0.7820, t=7061, NN=43)

In [5]:
lang = "latin"
# Load models
model1, model2 = get_models(lang)
# Load binary truths
binary_truth = get_gt(lang)
# Task 1 - Binary Classification
table = []
predictions = []
i = 0
for word in binary_truth[:, 0]:
    prediction = (
        0
        if moving_lncs2(word, model1, model2, 43, 0.7061) >= 0.7820
        else 1
    )
    predictions.append(prediction)
    table.append([word, str(binary_truth[i, 1]), str(prediction)])
    i += 1
print(tabulate(table, headers=["Word","Truth", "Prediction"]))
print("CLassification score for " + lang)
print(
    "\n"
    + classification_report(
        binary_truth[:, 1].astype(float),
        numpy.array(predictions),
        target_names=["class 0 (stable)", "class 1 (change)"],
    )
)
# Load scores truths
score_truth = get_gt(lang, binary=False)
table = []
# Task 2 - Semantic Shift Score
scores = []
i = 0
for word in score_truth[:, 0]:
    score = 1 - moving_lncs2(word, model1, model2, 43, 0.7061)
    scores.append(score)
    table.append([word, str(score_truth[i, 1]), str(score)])
    i += 1
print(tabulate(table, headers=["Word","Truth", "Rank"]))
rho, _ = spearmanr(scores, score_truth[:, 1], nan_policy="raise")
print("CLassification score for " + lang)
print("Spearman score for " + lang + ": " + str(rho))

Word           Truth    Prediction
-----------  -------  ------------
acerbus            0             0
adsumo             1             1
ancilla            0             1
beatus             1             1
civitas            1             1
cohors             1             1
consilium          0             0
consul             1             1
credo              1             1
dolus              1             1
dubius             1             1
dux                1             1
fidelis            0             1
honor              0             0
hostis             0             0
humanitas          1             1
imperator          1             1
itero              0             1
jus                1             1
licet              1             1
necessarius        0             1
nepos              1             1
nobilitas          0             0
oportet            0             1
poena              0             0
pontifex           1             1
potestas           1

### Swedish (Hyper on ACC: thr=0.5539, t=0.2343, NN=10)

In [6]:
lang = "swedish"
# Load models
model1, model2 = get_models(lang)
# Load binary truths
binary_truth = get_gt(lang)
# Task 1 - Binary Classification
table = []
predictions = []
i = 0
for word in binary_truth[:, 0]:
    prediction = (
        0
        if moving_lncs2(word, model1, model2, 10, 0.2343) >= 0.5539
        else 1
    )
    predictions.append(prediction)
    table.append([word, str(binary_truth[i, 1]), str(prediction)])
    i += 1
print(tabulate(table, headers=["Word","Truth", "Prediction"]))
print("CLassification score for " + lang)
print(
    "\n"
    + classification_report(
        binary_truth[:, 1].astype(float),
        numpy.array(predictions),
        target_names=["class 0 (stable)", "class 1 (change)"],
    )
)
# Load scores truths
score_truth = get_gt(lang, binary=False)
table = []
# Task 2 - Semantic Shift Score
scores = []
i = 0
for word in score_truth[:, 0]:
    score = 1 - moving_lncs2(word, model1, model2, 10, 0.2343)
    scores.append(score)
    table.append([word, str(score_truth[i, 1]), str(score)])
    i += 1
print(tabulate(table, headers=["Word","Truth", "Rank"]))
rho, _ = spearmanr(scores, score_truth[:, 1], nan_policy="raise")
print("CLassification score for " + lang)
print("Spearman score for " + lang + ": " + str(rho))

Word            Truth    Prediction
------------  -------  ------------
aktiv               0             0
annandag            0             0
antyda              0             1
bearbeta            0             0
bedömande           0             0
beredning           0             0
blockera            0             0
bolagsstämma        0             0
bröllop             0             0
by                  0             0
central             0             1
färg                0             0
förhandling         0             0
gagn                0             0
granskare           1             0
kemisk              0             0
kokärt              0             0
konduktör           1             1
krita               1             0
ledning             1             0
medium              1             1
motiv               1             0
notis               0             0
studie              0             0
undertrycka         0             0
uppfattning         1       